##Titanic

In [1]:
!pip install psycopg2-binary

In [0]:
import psycopg2

In [3]:
import pandas as pd
url = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module2-sql-for-analysis/titanic.csv'
df = pd.read_csv(url)
df.Age = df.Age.astype('int')
df['Name'] = df['Name'].str.replace(r"[\"\',]", '')
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
Survived                   887 non-null int64
Pclass                     887 non-null int64
Name                       887 non-null object
Sex                        887 non-null object
Age                        887 non-null int64
Siblings/Spouses Aboard    887 non-null int64
Parents/Children Aboard    887 non-null int64
Fare                       887 non-null float64
dtypes: float64(1), int64(5), object(2)
memory usage: 55.5+ KB


In [0]:
def gen_row_tuples(df):
  rows = [] 
  for idx, vals in df.iterrows():
    row = [idx]
    for val in vals.values:
      row.append(val)
    row = tuple(row)
    rows.append(row)
  return rows

In [6]:
titanic_row_tuples = gen_row_tuples(df)
titanic_row_tuples[0]

(28, 1, 3, 'Miss. Ellen ODwyer', 'female', 24, 0, 0, 7.8792)

In [0]:
dbname = 'mhodwfds'
user = 'mhodwfds'
password = 'v8dvWGf-qAVfuz3f9042cAMpVX-7ZNMS'
host = 'raja.db.elephantsql.com'

pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

pg_curs = pg_conn.cursor()

In [0]:
pg_curs.execute("DROP TABLE IF EXISTS titanic")

create_titanic_table = """
CREATE TABLE titanic (
  index SERIAL PRIMARY KEY,
  survived INT,
  pclass INT,
  name VARCHAR(85),
  sex VARCHAR(10),
  age INT,
  sibs_spouse INT,
  par_child INT,
  fare REAL
)
"""

pg_curs.execute(create_titanic_table)

In [0]:
for row in titanic_row_tuples:
  insert_row = "INSERT INTO titanic VALUES" + str(row)
  pg_curs.execute(insert_row)
pg_conn.commit()

In [0]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM titanic')
pg_titanic = pg_curs.fetchall()

In [12]:
pg_titanic[:5]

[(0, 0, 3, 'Mr. Owen Harris Braund', 'male', 22, 1, 0, 7.25),
 (1,
  1,
  1,
  'Mrs. John Bradley (Florence Briggs Thayer) Cumings',
  'female',
  38,
  1,
  0,
  71.2833),
 (2, 1, 3, 'Miss. Laina Heikkinen', 'female', 26, 0, 0, 7.925),
 (3,
  1,
  1,
  'Mrs. Jacques Heath (Lily May Peel) Futrelle',
  'female',
  35,
  1,
  0,
  53.1),
 (4, 0, 3, 'Mr. William Henry Allen', 'male', 35, 0, 0, 8.05)]

In [15]:
#compare survival rates bw man and women:
query = "SELECT sex, ROUND(AVG(survived), 3) AS avg_survival_rate FROM titanic GROUP BY sex;"
pg_curs.execute(query)
pg_curs.fetchall()

[('female', Decimal('0.742')), ('male', Decimal('0.190'))]

In [16]:
#compare survival between pclass:
query = "SELECT pclass, ROUND(AVG(survived), 3) AS avg_survival_rate FROM titanic GROUP BY pclass;"
pg_curs.execute(query)
pg_curs.fetchall()

[(1, Decimal('0.630')), (3, Decimal('0.244')), (2, Decimal('0.473'))]

In [18]:
#compare survival across pclass and sex, and get the number in each category:
query = "SELECT pclass, sex, ROUND(AVG(survived), 3) AS avg_survival, COUNT(sex) FROM titanic GROUP BY pclass, sex;"
pg_curs.execute(query)
pg_curs.fetchall()

[(1, 'female', Decimal('0.968'), 94),
 (1, 'male', Decimal('0.369'), 122),
 (2, 'male', Decimal('0.157'), 108),
 (2, 'female', Decimal('0.921'), 76),
 (3, 'female', Decimal('0.500'), 144),
 (3, 'male', Decimal('0.137'), 343)]